# Calcul des matrices de transformations homogènes et du Jacobien d'un robot série quelconque
Auteur : Miguel Da Silva  
Date : 19/01/2022  
Mail : miguel.dasilva@centralesupelec.fr

Import de la bibliothèque de calcul symbolique

In [ ]:
from sympy import* 

Initialisation des paramètres d'affichage des résultats analytiques donnés par sympy

In [2]:
init_printing()

## Description de la géométrie du robot grâce aux paramètres de Denavit-Hartenberg (DH)

In [3]:
l1, l2, l3, l4, t, alpha = symbols("l1 l2 l3 l4 t alpha")
q1 = Function('q1')(t)
q2 = Function('q2')(t)
q3 = Function('q3')(t)
q4 = Function('q4')(t)
q = transpose(Matrix([ [q1,q2,q3,q4] ]))

Tableau de DH :  
  
Première colonne (a<sub>i</sub>) : translation du repère i par rapport au repère i-1 selon l'axe **x<sub>i</sub>**  
Deuxième colonne ($\alpha$<sub>i</sub>) : angle de rotation autour du repère i par rapport au repère i-1 autour de l'axe **x<sub>i-1</sub>**  
Troisième colonne (d<sub>i</sub>) : translation du repère i par rapport au repère i-1 selon l'axe **y<sub>i</sub>**  
Quatrième colonne ($\theta$<sub>i</sub>) : angle de rotation du repère i par rapport au repère i-1 autour de l'axe **z<sub>i-1</sub>**

In [4]:
DH = Matrix([[+0.0119, -pi/2, l1, q1], 
             [l2, 0, 0, q2-pi/2+alpha], 
             [l3, 0, 0, q3+pi/2-alpha],
             [l4, 0, 0, q4]])

pprint(DH)

⎡        -π                     ⎤
⎢0.0119  ───  l₁      q₁(t)     ⎥
⎢         2                     ⎥
⎢                               ⎥
⎢                             π ⎥
⎢  l₂     0   0   α + q₂(t) - ─ ⎥
⎢                             2 ⎥
⎢                               ⎥
⎢                              π⎥
⎢  l₃     0   0   -α + q₃(t) + ─⎥
⎢                              2⎥
⎢                               ⎥
⎣  l₄     0   0       q₄(t)     ⎦


Nombre d'axes du robot = n

In [5]:
n=4
k = shape(DH)[0]
print(k)

4


## Calcul des matrices de transformations homogènes

Calcul des matrices de transformations homogènes d'un repère i à un repère i+1 pour tous les i allant de 0 (repère initial) à n (repère final) : **M<sub>01</sub>**, **M<sub>12</sub>**, **M<sub>23</sub>**, ... , **M<sub>n-1 n</sub>** 

In [6]:
M=[]
for i in range(k):
    M.append(Matrix([ [cos(DH[i,3]), -sin(DH[i,3])*cos(DH[i,1]), sin(DH[i,3])*sin(DH[i,1]), DH[i,0]*cos(DH[i,3])],
               [sin(DH[i,3]), cos(DH[i,3])*cos(DH[i,1]), -cos(DH[i,3])*sin(DH[i,1]), DH[i,0]*sin(DH[i,3])],
               [0, sin(DH[i,1]), cos(DH[i,1]), DH[i,2]],
               [0, 0, 0, 1]
               ])
             )

M

C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Pro

⎡⎡cos(q₁(t))  0   -sin(q₁(t))  0.0119⋅cos(q₁(t))⎤  ⎡sin(α + q₂(t))   cos(α + q
⎢⎢                                              ⎥  ⎢                          
⎢⎢sin(q₁(t))  0   cos(q₁(t))   0.0119⋅sin(q₁(t))⎥  ⎢-cos(α + q₂(t))  sin(α + q
⎢⎢                                              ⎥, ⎢                          
⎢⎢    0       -1       0              l₁        ⎥  ⎢       0               0  
⎢⎢                                              ⎥  ⎢                          
⎣⎣    0       0        0               1        ⎦  ⎣       0               0  

₂(t))  0  l₂⋅sin(α + q₂(t)) ⎤  ⎡sin(α - q₃(t))  -cos(α - q₃(t))  0  l₃⋅sin(α -
                            ⎥  ⎢                                              
₂(t))  0  -l₂⋅cos(α + q₂(t))⎥  ⎢cos(α - q₃(t))  sin(α - q₃(t))   0  l₃⋅cos(α -
                            ⎥, ⎢                                              
       1          0         ⎥  ⎢      0                0         1          0 
                            ⎥  ⎢                   

Calcul de la matrice homogène **M<sub>0n</sub>** qui permet d'obetnir le Modèle géométrique direct du robot (MGD) :  
Soit P<sub>0</sub> la position cartésienne de l'effecteur dans le repère d'origine et P<sub>n</sub> la position cartésienne de l'effecteur dans le repère n, alors P<sub>0</sub> = **M<sub>0n</sub>**.P<sub>n</sub>  

In [7]:
M0n = M[0]
for i in range(1,k):
    M0n = M0n*M[i]
M0n = simplify(M0n)
M0n

⎡cos(q₂(t) + q₃(t) + q₄(t))⋅cos(q₁(t))  -sin(q₂(t) + q₃(t) + q₄(t))⋅cos(q₁(t))
⎢                                                                             
⎢sin(q₁(t))⋅cos(q₂(t) + q₃(t) + q₄(t))  -sin(q₂(t) + q₃(t) + q₄(t))⋅sin(q₁(t))
⎢                                                                             
⎢     -sin(q₂(t) + q₃(t) + q₄(t))            -cos(q₂(t) + q₃(t) + q₄(t))      
⎢                                                                             
⎣                  0                                      0                   

  -sin(q₁(t))  (1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos
                                                                              
  cos(q₁(t))   (1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos
                                                                              
       0                     l₁ + l₂⋅cos(α + q₂(t)) - l₃⋅sin(q₂(t) + q₃(t)) - 
                                                   

## Calcul de la matrice Jacobienne

### Calcul de la matrice Jacobienne des vitesses linéaires de l'effecteur 

Extraction des coordonnées de l'origine du repère n dans le repère 0, depuis la matrice **M<sub>0n</sub>**

In [8]:
Tx = M0n[0,3]
Ty = M0n[1,3]
Tz = M0n[2,3]

T = transpose(Matrix([ [Tx, Ty, Tz] ]))

simplify(T)

⎡(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t)
⎢                                                                             
⎢(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t)
⎢                                                                             
⎣              l₁ + l₂⋅cos(α + q₂(t)) - l₃⋅sin(q₂(t) + q₃(t)) - l₄⋅sin(q₂(t) +

 + q₄(t)) + 0.0119)⋅cos(q₁(t))⎤
                              ⎥
 + q₄(t)) + 0.0119)⋅sin(q₁(t))⎥
                              ⎥
 q₃(t) + q₄(t))               ⎦

Jacobienne des vitesses linéaires de l'effecteur

In [9]:
J_v = zeros(3,n)

for i in range(3):
    for j in range(n):
        J_v[i,j]=T[i].diff(q[j])

simplify(J_v)

⎡-(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t
⎢                                                                             
⎢(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t)
⎢                                                                             
⎣                                                     0                       

) + q₄(t)) + 0.0119)⋅sin(q₁(t))  1.0⋅(l₂⋅cos(α + q₂(t)) - l₃⋅sin(q₂(t) + q₃(t)
                                                                              
 + q₄(t)) + 0.0119)⋅cos(q₁(t))   1.0⋅(l₂⋅cos(α + q₂(t)) - l₃⋅sin(q₂(t) + q₃(t)
                                                                              
                                         -l₂⋅sin(α + q₂(t)) - l₃⋅cos(q₂(t) + q

) - l₄⋅sin(q₂(t) + q₃(t) + q₄(t)))⋅cos(q₁(t))  -1.0⋅(l₃⋅sin(q₂(t) + q₃(t)) + l
                                                                              
) - l₄⋅sin(q₂(t) + q₃(t) + q₄(t)))⋅sin(q₁(t))  -1.

### Calcul de la matrice Jacobienne des vitesses angulaires de l'effecteur

Extraction de la matrice de rotation **R<sub>0n</sub>** depuis la matrice **M<sub>0n</sub>**

In [15]:
R0n = M0n[0:3,0:3]

R0n_dot = R0n.diff(t)
S_omega_0n = R0n_dot*transpose(R0n) 

omega_x = -S_omega_0n[1,2]
omega_y = S_omega_0n[0,2]
omega_z = -S_omega_0n[0,1]

omega = transpose(Matrix([ [omega_x, omega_y, omega_z] ]))

simplify(omega)

⎡⎛  d           d           d        ⎞           ⎤
⎢⎜- ──(q₂(t)) - ──(q₃(t)) - ──(q₄(t))⎟⋅sin(q₁(t))⎥
⎢⎝  dt          dt          dt       ⎠           ⎥
⎢                                                ⎥
⎢ ⎛d           d           d        ⎞            ⎥
⎢ ⎜──(q₂(t)) + ──(q₃(t)) + ──(q₄(t))⎟⋅cos(q₁(t)) ⎥
⎢ ⎝dt          dt          dt       ⎠            ⎥
⎢                                                ⎥
⎢                   d                            ⎥
⎢                   ──(q₁(t))                    ⎥
⎣                   dt                           ⎦

Jacobienne des vitesses angulaires de l'effecteur  
**J_omega** est de la vitesse de rotation du repère n par rapport au repère 0 issue de la représentation angle/axe

In [21]:
J_omega = zeros(3,n)
q_dot = zeros(1,n)

for j in range(n):
    q_dot[j] = q[j].diff(t)
    
for i in range(3):
    for j in range(n):    
        J_omega[i,j]=omega[i].diff(q_dot[j])

print(J_omega)
simplify(J_omega)

Matrix([[0, -sin(q2(t) + q3(t) + q4(t))**2*sin(q1(t)) - sin(q1(t))*cos(q2(t) + q3(t) + q4(t))**2, -sin(q2(t) + q3(t) + q4(t))**2*sin(q1(t)) - sin(q1(t))*cos(q2(t) + q3(t) + q4(t))**2, -sin(q2(t) + q3(t) + q4(t))**2*sin(q1(t)) - sin(q1(t))*cos(q2(t) + q3(t) + q4(t))**2], [0, sin(q2(t) + q3(t) + q4(t))**2*cos(q1(t)) + cos(q2(t) + q3(t) + q4(t))**2*cos(q1(t)), sin(q2(t) + q3(t) + q4(t))**2*cos(q1(t)) + cos(q2(t) + q3(t) + q4(t))**2*cos(q1(t)), sin(q2(t) + q3(t) + q4(t))**2*cos(q1(t)) + cos(q2(t) + q3(t) + q4(t))**2*cos(q1(t))], [sin(q2(t) + q3(t) + q4(t))**2*sin(q1(t))**2 + sin(q1(t))**2*cos(q2(t) + q3(t) + q4(t))**2 + cos(q1(t))**2, 0, 0, 0]])


C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\louis\AppData\Local\Pro

⎡0  -sin(q₁(t))  -sin(q₁(t))  -sin(q₁(t))⎤
⎢                                        ⎥
⎢0  cos(q₁(t))   cos(q₁(t))   cos(q₁(t)) ⎥
⎢                                        ⎥
⎣1       0            0            0     ⎦

### Construction de la matrice Jacobienne

In [12]:
J = simplify(Matrix([ [J_v], [J_omega] ]))
print(J)
J

Matrix([[-(1.0*l2*sin(alpha + q2(t)) + 1.0*l3*cos(q2(t) + q3(t)) + 1.0*l4*cos(q2(t) + q3(t) + q4(t)) + 0.0119)*sin(q1(t)), 1.0*(l2*cos(alpha + q2(t)) - l3*sin(q2(t) + q3(t)) - l4*sin(q2(t) + q3(t) + q4(t)))*cos(q1(t)), -1.0*(l3*sin(q2(t) + q3(t)) + l4*sin(q2(t) + q3(t) + q4(t)))*cos(q1(t)), -1.0*l4*sin(q2(t) + q3(t) + q4(t))*cos(q1(t))], [(1.0*l2*sin(alpha + q2(t)) + 1.0*l3*cos(q2(t) + q3(t)) + 1.0*l4*cos(q2(t) + q3(t) + q4(t)) + 0.0119)*cos(q1(t)), 1.0*(l2*cos(alpha + q2(t)) - l3*sin(q2(t) + q3(t)) - l4*sin(q2(t) + q3(t) + q4(t)))*sin(q1(t)), -1.0*(l3*sin(q2(t) + q3(t)) + l4*sin(q2(t) + q3(t) + q4(t)))*sin(q1(t)), -1.0*l4*sin(q2(t) + q3(t) + q4(t))*sin(q1(t))], [0, -l2*sin(alpha + q2(t)) - l3*cos(q2(t) + q3(t)) - l4*cos(q2(t) + q3(t) + q4(t)), -l3*cos(q2(t) + q3(t)) - l4*cos(q2(t) + q3(t) + q4(t)), -l4*cos(q2(t) + q3(t) + q4(t))], [0, -sin(q1(t)), -sin(q1(t)), -sin(q1(t))], [0, cos(q1(t)), cos(q1(t)), cos(q1(t))], [1, 0, 0, 0]])


⎡-(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t
⎢                                                                             
⎢(1.0⋅l₂⋅sin(α + q₂(t)) + 1.0⋅l₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l₄⋅cos(q₂(t) + q₃(t)
⎢                                                                             
⎢                                                     0                       
⎢                                                                             
⎢                                                     0                       
⎢                                                                             
⎢                                                     0                       
⎢                                                                             
⎣                                                     1                       

) + q₄(t)) + 0.0119)⋅sin(q₁(t))  1.0⋅(l₂⋅cos(α + q₂(t)) - l₃⋅sin(q₂(t) + q₃(t)
                                                   